# 基于MindSpore构造非对称损失函数

 本实验主要介绍非对称损失函数的原理和构造，使用MindSpore构造非对称损失函数，以Focal Loss损失函数作为讲解实例。

## 1、实验目的

- 掌握非对称损失函数的原理。
- 掌握如何使用MindSpore构造非对称损失函数。

## 2、非对称损失函数原理介绍

非对称损失函数是一种最优化方法，通过控制参数变化，使模型获得适合的拟合行为，从而降低误差和达到最优效果。它是在构建模型权重时，从损失函数中提取出来的一种损失函数，它可以调整模型拟合程度，改善模型预测精确度。

Focal Loss函数：Focal Loss是基于二分类交叉熵损失的。它是一个动态缩放的交叉熵损失，通过一个动态缩放因子，可以动态降低训练过程中易区分样本的权重，从而将重心快速聚焦在那些难区分的样本（有可能是正样本，也有可能是负样本，但都是对训练网络有帮助的样本），公式如下：
$$FL(p_t)=-{{\alpha}_t}(1-p_t)^{\gamma}log(p_t)$$
即通过${{\alpha}_t}$可以抑制正负样本的数量失衡，通过${\gamma}$可以控制简单/难区分样本数量失衡。

${\gamma}$为一个参数，范围在[0,5]， 当${\gamma}$为0时，就变成了二分类交叉熵损失函数。$(1-p_t)^{\gamma}$可以降低易分样本的损失贡献，从而增加难分样本的损失比例。当$p_t$趋向于1，即说明该样本是易区分样本，此时调制因子$(1-p_t)^{\gamma}$是趋向于0，说明对损失的贡献较小，即降低了易区分样本的损失比例；当$p_t$很小，也就是假如某个样本被分到正样本，但是该样本为前景的概率特别小，即被错分到正样本了，此时调制因子$(1-p_t)^{\gamma}$是趋向于1，对loss也没有太大的影响。

Focal Loss的特点：

（1）调制因子$(1-p_t)^{\gamma}$是用来减低易分样本的损失贡献 ，无论是前景类还是背景类，$p_t$越大，就说明该样本越容易被区分，调制因子也就越小。

（2）${{\alpha}_t}$用于调节正负样本损失之间的比例，前景类别使用${{\alpha}_t}$时，对应的背景类别使用$1-{{\alpha}_t}$。

（3）${\gamma}$和${{\alpha}_t}$都有相应的取值范围，他们的取值相互间也是有影响的，在实际使用过程中应组合使用。

## 3、实验环境

在动手进行实践之前，需要注意以下几点：
* 确保实验环境正确安装，包括安装MindSpore。安装过程：首先登录[MindSpore官网安装页面](https://www.mindspore.cn/install)，根据安装指南下载安装包及查询相关文档。同时，官网环境安装也可以按下表说明找到对应环境搭建文档链接，根据环境搭建手册配置对应的实验环境。
* 推荐使用交互式的计算环境Jupyter Notebook，其交互性强，易于可视化，适合频繁修改的数据分析实验环境。
* 实验也可以在华为云一站式的AI开发平台ModelArts上完成。
* 推荐实验环境：MindSpore版本=MindSpore 2.0；Python环境=3.7


|  硬件平台 |  操作系统  | 软件环境 | 开发环境 | 环境搭建链接 |
| :-----:| :----: | :----: |:----:   |:----:   |
| CPU | Windows-x64 | MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.1节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| GPU CUDA 10.1|Linux-x86_64| MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.2节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| Ascend 910  | Linux-x86_64| MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第四章](./MindSpore环境搭建实验手册.docx)|

## 4、数据处理

### 4.1 数据准备

在本次实验中，我们使用numpy从均匀分布中随机生成测试数据，从正态分布中随机生成噪声加入因变量y1和y2。

### 4.2 数据加载

使用GeneratorDataset，通过迭代列表构造数据集，指定生成数据集的列名为data和lable，使用batch函数指定每个批处理数据包含的数据条目。

In [24]:
# 导入必要的库，包括NumPy和MindSpore
import numpy as np
from mindspore import dataset as ds
import mindspore.nn as nn
import mindspore as ms

# 生成带有两个标签的数据集
def get_multilabel_data(num, w=2.0, b=3.0):
    for _ in range(num):
        x = np.random.uniform(-10.0, 10.0) # 生成服从(-10.0, 10.0)范围内的均匀分布的元素，返回值的元素类型为浮点型。
        # noise1和noise2为服从标准正态分布的随机值
        noise1 = np.random.normal(0, 1) # 随机产生一个服从正态分布(0,1)的数值
        noise2 = np.random.normal(-1, 1)
        # 定义第一个标签
        y1 = x * w + b + noise1 # 增加噪音生成y                    
        # 定义第二个标签
        y2 = x * w + b + noise2                   
        yield np.array([x]).astype(np.float32), np.array([y1]).astype(np.float32), np.array([y2]).astype(np.float32)
# 定义了一个生成带有两个标签的数据集的函数get_multilabel_data。
# 函数的参数num表示要生成的数据数量，w和b是用于计算标签的参数。
# 在函数内部，使用NumPy生成随机数来生成输入数据x，以及服从标准正态分布的随机噪声noise1和noise2。
# 然后，根据给定的公式计算两个标签y1和y2。最后，使用yield语句返回每个数据样本的输入和两个标签。

def create_multilabel_dataset(num_data, batch_size=16):
    # 加载数据集eval_data=list(get_multilabel_data(5))
    train_data=list(get_multilabel_data(num_data))
    dataset = ds.GeneratorDataset(list(get_multilabel_data(num_data)), column_names=['data', 'label1', 'label2'])
    # 每个batch有16个数据
    dataset = dataset.batch(batch_size) 
    return dataset

# 可视化生成的数据
eval_data=list(get_multilabel_data(5))
x,y1,y2=zip(*eval_data) # zip()函数迭代eval_data，将eval_data中的元素打包成一个个元组，然后返回由这些元组组成的列表。
eval_data

[(array([-5.8620415], dtype=float32),
  array([-10.054771], dtype=float32),
  array([-8.221005], dtype=float32)),
 (array([5.6895947], dtype=float32),
  array([13.611581], dtype=float32),
  array([11.341168], dtype=float32)),
 (array([3.3639672], dtype=float32),
  array([8.234134], dtype=float32),
  array([8.281881], dtype=float32)),
 (array([-9.866455], dtype=float32),
  array([-16.06036], dtype=float32),
  array([-18.147293], dtype=float32)),
 (array([1.7907822], dtype=float32),
  array([7.481146], dtype=float32),
  array([5.612006], dtype=float32))]

## 5、模型构建

### 5.1 导入Python库和模块

在使用前，导入需要的Python库和模块。

In [25]:
# 时间处理模块
import time
# 科学计算库
import numpy as np
# MindSpore库
import mindspore as ms
# 常见算子操作
import mindspore.ops as ops
# 数据集处理模块
from mindspore import dataset as ds
# 神经网络模块，张量，模型编译
from mindspore import Tensor
# 模型训练设置
from mindspore.train import Callback, LossMonitor, Model
# L1型损失函数
from mindspore.nn import L1Loss
# MindSpore环境设置的0号种子
ms.common.set_seed(0)
import mindspore.nn as nn
from mindspore.common.initializer import Normal

### 5.2 定义模型

定义线性回归网络和构造函数。

In [26]:
# 定义线性回归网络
class LinearNet(nn.Cell):
    def __init__(self):
        super(LinearNet, self).__init__()
        self.fc = nn.Dense(1, 1, Normal(0.02), Normal(0.02))

    def construct(self, x):
        return self.fc(x)

### 5.3 自定义Focal Loss损失函数

首先定义多标签损失函数，再自定义Focal Loss损失函数。

In [27]:
# 定义多标签损失函数
class MAELossForMultiLabel(nn.LossBase):
    def __init__(self, reduction="mean"):
        super(MAELossForMultiLabel, self).__init__(reduction)
        self.abs = ops.abs

    def construct(self, base, target1, target2):
        # 计算第一个标签的误差
        x1 = self.abs(base - target1)
        # 计算第二个标签的误差
        x2 = self.abs(base - target2)
        # 将两误差取平均后作为最终的损失函数值                           
        return (self.get_loss(x1) + self.get_loss(x2))/2   
    
# 自定义Focal Loss损失函数
class FocalLoss(nn.Cell):
    def __init__(self, backbone, loss_fn):
        super(FocalLoss, self).__init__(auto_prefix=False)
        self._backbone = backbone
        self._loss_fn = loss_fn

    def construct(self, data, label1, label2):
        output = self._backbone(data)
        return self._loss_fn(output, label1, label2)

## 6、模型训练

完成数据预处理、网络定义、损失函数之后，选择优化器，开始模型训练。模型训练包含10层迭代，数据集按分组从训练集中抽取数据，输入网络计算得到损失函数。

In [28]:
ds_train = create_multilabel_dataset(num_data=160)
net = LinearNet()
# 定义多标签损失函数
loss = MAELossForMultiLabel()
# 定义损失网络，连接前向网络和多标签损失函数
loss_net = FocalLoss(net, loss)
# 定义优化器
opt = nn.Momentum(net.trainable_params(), learning_rate=0.005, momentum=0.9)
# 定义Model，多标签场景下Model无需指定损失函数
model = ms.train.Model(network=loss_net, optimizer=opt)
model.train(epoch=10, train_dataset=ds_train, callbacks=[LossMonitor(5)])

epoch: 1 step: 5, loss is 7.997321128845215
epoch: 1 step: 10, loss is 6.678524971008301
epoch: 2 step: 5, loss is 2.5805487632751465
epoch: 2 step: 10, loss is 3.0239973068237305
epoch: 3 step: 5, loss is 2.7806477546691895
epoch: 3 step: 10, loss is 2.1242778301239014
epoch: 4 step: 5, loss is 2.6970858573913574
epoch: 4 step: 10, loss is 2.183283567428589
epoch: 5 step: 5, loss is 2.101006031036377
epoch: 5 step: 10, loss is 1.4374243021011353
epoch: 6 step: 5, loss is 1.5233697891235352
epoch: 6 step: 10, loss is 1.1681139469146729
epoch: 7 step: 5, loss is 1.2294151782989502
epoch: 7 step: 10, loss is 0.93113112449646
epoch: 8 step: 5, loss is 0.7108897566795349
epoch: 8 step: 10, loss is 0.8908092975616455
epoch: 9 step: 5, loss is 0.8534233570098877
epoch: 9 step: 10, loss is 1.207047700881958
epoch: 10 step: 5, loss is 0.7830170392990112
epoch: 10 step: 10, loss is 0.867098331451416


## 7、模型预测

首先随机生成测试数据，再定义两个标签，然后进行模型测试，最后输出模型的预测结果。

In [29]:
from mindspore.train import Model
from mindspore import Tensor

model_predict = Model(net,loss_net,opt,metrics={"loss"})
# 生成测试数据
w=2.0
b=3.0
x = np.random.uniform(-10.0, 10.0, (1,1))
x1 = np.array([x]).astype(np.float32)
# 定义第一个标签
true_result1 = x * w + b      
# 定义第二个标签
true_result2 = x * w + b            
print('data:' + '%s'%x)
# 模型测试
test_result = model_predict.predict(Tensor(x1))
print('predict result:' + '%s'%test_result)
print('true result1:' + '%s'%true_result1)
print('true result2:' + '%s'%true_result2)

data:[[1.81170758]]
predict result:[[[5.9306245]]]
true result1:[[6.62341516]]
true result2:[[6.62341516]]
